# Compare PyApriori with other Python packages

Packages:
- mlxtend.frequent_patterns.apriori
- PyFIM
- apyori
- pyapriori (this package)

Data:
- accidents
- connect
- chess
- mushroom

Source: Frequent Itemset Mining Implementations Repository (http://fimi.uantwerpen.be/data/) 

# New package

In [1]:
def add_results(results, candidates, candidates_support):
    results += [(candidate, candidates_support[i]) for i, candidate in enumerate(candidates)]
    return results

In [2]:
def get_candidates_tables(reduced_table, reduced_features, results, min_length, min_support = 0, stop_candidates = []):
    table_list = []
    candidates_list = []
    support_list = []
    for i, element in enumerate(reduced_features[:-1]):
        print(element)
        print(reduced_features[i+1:])
        
        candidates = element | reduced_features[i+1:]
        print(candidates)
        print('---------------')
        
        candidates_mask = [candidate not in stop_candidates for candidate in candidates]
        final_candidates = candidates[candidates_mask]
        mask = reduced_table[:,i]
        table = reduced_table[mask,:][:,i+1:][:,candidates_mask]
        candidates_support = table.sum(axis=0)
        support_mask = candidates_support >= min_support
        
        stop_candidates = stop_candidates + list(final_candidates[np.invert(support_mask)])
        table = table[:,support_mask]
        final_candidates = final_candidates[support_mask]
        candidates_support = candidates_support[support_mask]
        
        support_order = np.argsort(candidates_support)
        table = table[:, support_order]
        final_candidates = final_candidates[support_order]
        candidates_support = candidates_support[support_order]
        
        table_list.append(table)
        candidates_list.append(final_candidates)
        support_list.append(candidates_support)
        
        if len(final_candidates)>0 and len(final_candidates[0]) >= min_length:
            results = add_results(results, final_candidates, candidates_support)
        
    return table_list, candidates_list, support_list, stop_candidates, results

In [3]:
def apriori(transactions, min_support, min_length=0):
    results = []
    k = 1
    
    support = transactions.sum(axis=0)
    support_order = np.argsort(support)
    support_sorted = support[support_order]
    min_support_mask = support_sorted > min_support
    features_list = support_order[min_support_mask]
    features_list = np.array([set([v]) for v in list(features_list)])
    
    features_support = support_sorted[min_support_mask]
    transactions = transactions[:,support_order][:,min_support_mask]
    
    if k >= min_length:
        results = add_results(results, features_list, features_support)
    
    tree = []
    tree.append([[transactions], [features_list]])
    stop_candidates = []
    
    while len(tree) > 0:
        transactions, features_list = tree.pop()
        for i, table in enumerate(transactions):
            table_list, final_candidates, support_list, stop_candidates, results = get_candidates_tables(table, features_list[i], results, min_length, min_support, stop_candidates)
            if len(final_candidates) > 0:
                tree.insert(0,[table_list, final_candidates])      
    return results

## Data Preparation

In [4]:
import pandas as pd
import numpy as np
import csv
from mlxtend.preprocessing import TransactionEncoder

Load datasets

In [5]:
dataset = [['chleba', 'mliko'], ['chleba', 'rohliky'], ['rohliky', 'mliko'], ['mliko', 'rohliky', 'chleba']]

all_features = set()
for tr in dataset:
    trset = set(tr)
    all_features = all_features.union(trset)
allf = list(all_features)  

data = []
for feat in allf:
    print(feat)
    dataf = []
    for i, tr in enumerate(dataset):
        if feat in tr:
            dataf.append(i)
    data.append(set(dataf))
data = np.array(data)
    
data

mliko
rohliky
chleba


array([{0, 2, 3}, {1, 2, 3}, {0, 1, 3}], dtype=object)

In [6]:
datamatrix = np.array([[1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]])
apriori(datamatrix,2)

{0}
[{1} {2}]
[{0, 1} {0, 2}]
---------------
{1}
[{2}]
[{1, 2}]
---------------


[({0}, 3), ({1}, 3), ({2}, 3), ({0, 2}, 3), ({1, 2}, 3)]

In [7]:
def load_dataset(datafile):
    # pyapriori
    dataset = [i.strip().split() for i in open(datafile).readlines()]
    all_features = set()
    for tr in dataset:
        trset = set(tr)
        all_features = all_features.union(trset)
    allf = list(all_features)  

    data = []
    for feat in allf:
        dataf = []
        for i, tr in enumerate(dataset):
            if feat in tr:
                dataf.append(i)
        data.append(set(dataf))
    data = np.array(data)
    
    
    #dataset = [i.strip().split() for i in open(datafile).readlines()]
    # Number of transactions
    transactions_count = len(dataset)
    #One Hot Encoding
    te = TransactionEncoder()
    te_dummies = te.fit(dataset).transform(dataset)
    # Dataframe with One Hot Encoding
    df = pd.DataFrame(te_dummies, columns=te.columns_)
    datamatrix = df.to_numpy()
    # CSC Matrix with One Hot Encoding

    
    
    return data, dataset, transactions_count, datamatrix.astype(bool)

## Modeling

In [8]:
import time
from mlxtend.frequent_patterns import apriori as mlxtend_apriori
import fim
from apyori import apriori as apyori_apriori
from pyapriori import PyApriori
from interruptingcow import timeout
from efficient_apriori import apriori as efapriori

The model fit is killed in 30s.

In [9]:
def run_models(minsup, data, dataset, transactions_count, datamatrix):
    
    print("")
    print("new")
    try:
        start = time.time()
        with timeout(30, exception=RuntimeError):
            result = apriori(datamatrix,round(transactions_count * (minsup/100)), 1)
        end = time.time()
        new_t = end - start
        new_count = len(result)  
    except Exception as inst:
        print(inst)
        print(inst.args)  
        new_t = None
        new_count = None 
    print('new apriori Done in ' + str(new_t) + 's')    

    print("")
    print("pyapriori")
    try:
        start = time.time()
        with timeout(30, exception=RuntimeError):
            py_apriori = PyApriori(round(transactions_count * (minsup/100)), 1)
            _, result = py_apriori.fit(data)
        end = time.time()
        pyapriori_t = end - start
        pyapriori_count = len(result)  
    except Exception as inst:
        print(inst)
        print(inst.args)  
        pyapriori_t = None
        pyapriori_count = None 
    print('pyapriori Done in ' + str(pyapriori_t) + 's')    
    
    print("")
    print("fim")
    try:
        start = time.time()
        with timeout(30, exception=RuntimeError):
            result = fim.apriori(dataset, supp=minsup)
        end = time.time()
        fim_t = end - start
        fim_count = len(result)  
    except Exception as inst:
        print(inst)
        print(inst.args)  
        fim_t = None
        fim_count = None 
    print('fim Done in ' + str(fim_t) + 's')    
    
    print("")
    print("apyori")
    try:
        #raise NameError('HiThere')
        start = time.time()
        with timeout(30, exception=RuntimeError):
            result = list(apyori_apriori(dataset, min_support=(minsup/100)))
        end = time.time()
        apyori_t = end - start
        apyori_count = len(result)
    except Exception as inst:
        print(inst)
        print(inst.args)  
        apyori_t = None
        apyori_count = None
    print('apyori Done in ' + str(apyori_t) + 's')   
    
    print("")
    print("effecient apriori")
    try:
        #raise NameError('HiThere')
        start = time.time()
        with timeout(30, exception=RuntimeError):
            result = list(efapriori(dataset, min_support=(minsup/100))[0])
        end = time.time()
        efapriori_t = end - start
        efapriori_count = len(result)
    except Exception as inst:
        print(inst)
        print(inst.args)  
        efapriori_t = None
        efapriori_count = None
    print('efapriori Done in ' + str(apyori_t) + 's')   
        
    
    return fim_t, fim_count, apyori_t, apyori_count, pyapriori_t, pyapriori_count, new_t, new_count,

## Run test

Parameters

In [10]:
supports = [40, 50]#, 60, 70
datafiles = ['data/mushroom.dat', 'data/accidents.dat', 'data/chess.dat'] #'data/accidents.dat', 'data/chess.dat', 

Test

In [11]:
#import cupy as cp
#cp.cuda.set_allocator(cp.cuda.MemoryPool(cp.cuda.malloc_managed).malloc)
#cp.cuda.set_allocator(cp.cuda.MemoryPool().malloc)
test_result = {}
for minsup in supports:
    for datafile in datafiles:
        print((minsup, datafile))
        data, dataset, transactions_count, datamatrix = load_dataset(datafile)
        fim_t, fim_count, apyori_t, apyori_count, pyapriori_t, pyapriori_count, new_t, new_count = run_models(minsup, data, dataset, transactions_count, datamatrix)
        test_result[(minsup, datafile)] = {'fim_t':fim_t, 'fim_count':fim_count, 'apyori_t':apyori_t, 'apyori_count':apyori_count, 'pyapriori_t':pyapriori_t, 'pyapriori_count':pyapriori_count, 'new_t':new_t, 'new_count':new_count }

(40, 'data/mushroom.dat')

new
{35}
[{67} {40} {42} {71} {0} {112} {13} {31} {93} {83} {68} {36} {79} {74}
 {52} {49} {109} {47} {104} {103}]
[{67, 35} {40, 35} {42, 35} {35, 71} {0, 35} {112, 35} {35, 13} {35, 31}
 {35, 93} {83, 35} {35, 68} {35, 36} {35, 79} {74, 35} {35, 52} {49, 35}
 {35, 109} {35, 47} {104, 35} {35, 103}]
---------------
{67}
[{40} {42} {71} {0} {112} {13} {31} {93} {83} {68} {36} {79} {74} {52}
 {49} {109} {47} {104} {103}]
[{40, 67} {42, 67} {67, 71} {0, 67} {112, 67} {67, 13} {67, 31} {67, 93}
 {83, 67} {67, 68} {67, 36} {67, 79} {74, 67} {67, 52} {49, 67} {67, 109}
 {67, 47} {104, 67} {67, 103}]
---------------
{40}
[{42} {71} {0} {112} {13} {31} {93} {83} {68} {36} {79} {74} {52} {49}
 {109} {47} {104} {103}]
[{40, 42} {40, 71} {40, 0} {40, 112} {40, 13} {40, 31} {40, 93} {40, 83}
 {40, 68} {40, 36} {40, 79} {40, 74} {40, 52} {40, 49} {40, 109} {40, 47}
 {40, 104} {40, 103}]
---------------
{42}
[{71} {0} {112} {13} {31} {93} {83} {68} {36} {79} {74} {52} {49

apyori Done in 1.8503704071044922s

effecient apriori
efapriori Done in 1.8503704071044922s
(40, 'data/accidents.dat')

new
{447}
[{417} {425} {178} {322} {400} {145} {223} {1} {311} {45} {428} {134} {0}
 {446} {423} {167} {345} {200} {56} {156} {189} {367} {211} {123} {234}
 {67} {89} {23} {78}]
[{417, 447} {425, 447} {178, 447} {322, 447} {400, 447} {145, 447}
 {223, 447} {1, 447} {311, 447} {45, 447} {428, 447} {134, 447} {0, 447}
 {446, 447} {423, 447} {167, 447} {345, 447} {200, 447} {56, 447}
 {156, 447} {189, 447} {367, 447} {211, 447} {123, 447} {234, 447}
 {67, 447} {89, 447} {23, 447} {78, 447}]
---------------
{417}
[{425} {178} {322} {400} {145} {223} {1} {311} {45} {428} {134} {0} {446}
 {423} {167} {345} {200} {56} {156} {189} {367} {211} {123} {234} {67}
 {89} {23} {78}]
[{417, 425} {417, 178} {417, 322} {400, 417} {417, 145} {417, 223}
 {417, 1} {417, 311} {417, 45} {417, 428} {417, 134} {0, 417} {417, 446}
 {417, 423} {417, 167} {345, 417} {200, 417} {56, 417} {417, 15

{322, 123}
[{322, 234} {322, 67} {89, 322} {322, 23} {322, 78}]
[{234, 322, 123} {67, 322, 123} {89, 322, 123} {322, 123, 23}
 {322, 123, 78}]
---------------
{322, 234}
[{322, 67} {89, 322} {322, 23} {322, 78}]
[{67, 322, 234} {89, 322, 234} {322, 234, 23} {322, 234, 78}]
---------------
{322, 67}
[{89, 322} {322, 23} {322, 78}]
[{89, 322, 67} {322, 67, 23} {322, 67, 78}]
---------------
{89, 322}
[{322, 23} {322, 78}]
[{89, 322, 23} {89, 322, 78}]
---------------
{322, 23}
[{322, 78}]
[{322, 78, 23}]
---------------
{400, 189}
[{400, 156} {400, 56} {400, 200} {400, 123} {400, 211} {400, 367}
 {400, 234} {400, 67} {400, 89} {400, 23} {400, 78}]
[{400, 156, 189} {400, 56, 189} {400, 189, 200} {400, 123, 189}
 {400, 211, 189} {400, 189, 367} {400, 234, 189} {400, 67, 189}
 {400, 89, 189} {400, 189, 23} {400, 189, 78}]
---------------
{400, 156}
[{400, 56} {400, 200} {400, 123} {400, 211} {400, 367} {400, 234}
 {400, 67} {400, 89} {400, 23} {400, 78}]
[{400, 56, 156} {400, 156, 200} {400

{446, 311}
[{167, 311} {423, 311} {200, 311} {345, 311} {56, 311} {156, 311}
 {189, 311} {367, 311} {211, 311} {123, 311} {234, 311} {67, 311}
 {89, 311} {23, 311} {78, 311}]
[{167, 446, 311} {423, 446, 311} {200, 446, 311} {345, 446, 311}
 {56, 446, 311} {156, 446, 311} {189, 446, 311} {367, 446, 311}
 {211, 446, 311} {123, 446, 311} {234, 446, 311} {67, 446, 311}
 {89, 446, 311} {23, 446, 311} {78, 446, 311}]
---------------
{167, 311}
[{423, 311} {200, 311} {345, 311} {56, 311} {156, 311} {189, 311}
 {367, 311} {211, 311} {123, 311} {234, 311} {67, 311} {89, 311} {23, 311}
 {78, 311}]
[{167, 423, 311} {200, 167, 311} {345, 167, 311} {56, 167, 311}
 {167, 156, 311} {167, 189, 311} {167, 367, 311} {167, 211, 311}
 {167, 123, 311} {167, 234, 311} {167, 67, 311} {89, 167, 311}
 {167, 23, 311} {167, 78, 311}]
---------------
{423, 311}
[{200, 311} {345, 311} {56, 311} {156, 311} {189, 311} {367, 311}
 {211, 311} {123, 311} {234, 311} {67, 311} {89, 311} {23, 311} {78, 311}]
[{200, 423, 3

{200, 428}
[{56, 428} {345, 428} {156, 428} {428, 189} {428, 367} {123, 428}
 {211, 428} {234, 428} {67, 428} {89, 428} {428, 23} {428, 78}]
[{200, 56, 428} {200, 345, 428} {200, 156, 428} {200, 428, 189}
 {200, 428, 367} {200, 123, 428} {200, 211, 428} {200, 234, 428}
 {200, 67, 428} {200, 89, 428} {200, 428, 23} {200, 428, 78}]
---------------
{56, 428}
[{345, 428} {156, 428} {428, 189} {428, 367} {123, 428} {211, 428}
 {234, 428} {67, 428} {89, 428} {428, 23} {428, 78}]
[{56, 345, 428} {56, 156, 428} {56, 428, 189} {56, 428, 367}
 {56, 123, 428} {56, 211, 428} {56, 234, 428} {56, 67, 428} {56, 89, 428}
 {56, 428, 23} {56, 428, 78}]
---------------
{345, 428}
[{156, 428} {428, 189} {428, 367} {123, 428} {211, 428} {234, 428}
 {67, 428} {89, 428} {428, 23} {428, 78}]
[{345, 428, 156} {345, 428, 189} {345, 428, 367} {345, 123, 428}
 {345, 211, 428} {345, 234, 428} {345, 67, 428} {89, 345, 428}
 {345, 428, 23} {345, 428, 78}]
---------------
{156, 428}
[{428, 189} {428, 367} {123, 428} 

...

{234, 367, 23, 311, 89, 123}
[{234, 78, 367, 311, 89, 123}]
[{234, 78, 367, 23, 89, 123, 311}]
---------------
{67, 367, 23, 311, 89, 123}
[{67, 78, 367, 311, 89, 123}]
[{67, 78, 367, 23, 89, 123, 311}]
---------------
{67, 234, 367, 23, 311, 89}
[{67, 234, 78, 367, 311, 89}]
[{67, 234, 78, 367, 23, 89, 311}]
---------------
{67, 234, 211, 311, 89, 123}
[{67, 234, 211, 23, 311, 123} {67, 234, 78, 211, 311, 123}]
[{67, 234, 211, 23, 311, 89, 123} {67, 234, 78, 211, 311, 89, 123}]
---------------
{67, 234, 211, 23, 311, 123}
[{67, 234, 78, 211, 311, 123}]
[{67, 234, 78, 211, 23, 123, 311}]
---------------
{234, 211, 23, 311, 89, 123}
[{234, 78, 211, 311, 89, 123}]
[{234, 78, 211, 23, 89, 123, 311}]
---------------
{67, 211, 23, 311, 89, 123}
[{67, 78, 211, 311, 89, 123}]
[{67, 78, 211, 23, 89, 123, 311}]
---------------
{67, 234, 211, 23, 311, 89}
[{67, 234, 78, 211, 311, 89}]
[{67, 234, 78, 211, 23, 89, 311}]
---------------
{67, 234, 23, 311, 89, 123}
[{67, 234, 78, 311, 89, 123}]
[{67

---------------
{67, 45, 367, 23, 56, 345}
[{67, 45, 78, 367, 56, 345}]
[{67, 45, 78, 367, 23, 56, 345}]
---------------
{345, 45, 367, 23, 56, 89}
[{345, 45, 78, 367, 56, 89}]
[{345, 45, 78, 367, 23, 56, 89}]
---------------
{89, 67, 234, 45, 367, 345}
[{67, 234, 45, 367, 23, 345} {67, 234, 45, 78, 367, 345}]
[{89, 67, 234, 45, 367, 23, 345} {89, 67, 234, 45, 78, 367, 345}]
---------------
{67, 234, 45, 367, 23, 345}
[{67, 234, 45, 78, 367, 345}]
[{67, 234, 45, 78, 367, 23, 345}]
---------------
{89, 234, 45, 367, 23, 345}
[{89, 234, 45, 78, 367, 345}]
[{89, 234, 45, 78, 367, 23, 345}]
---------------
{89, 67, 45, 367, 23, 345}
[{89, 67, 45, 78, 367, 345}]
[{89, 67, 45, 78, 367, 23, 345}]
---------------
{89, 67, 45, 56, 345, 123}
[{67, 45, 23, 56, 345, 123} {67, 45, 78, 56, 345, 123}]
[{67, 345, 45, 23, 56, 89, 123} {67, 345, 45, 78, 56, 89, 123}]
---------------
{67, 45, 23, 56, 345, 123}
[{67, 45, 78, 56, 345, 123}]
[{67, 45, 78, 23, 56, 345, 123}]
---------------
{345, 45, 23, 56,

{67, 234, 45, 367, 89, 156}
[{67, 234, 45, 367, 23, 156} {67, 234, 45, 78, 367, 156}]
[{67, 234, 45, 367, 23, 89, 156} {67, 234, 45, 78, 367, 89, 156}]
---------------
{67, 234, 45, 367, 23, 156}
[{67, 234, 45, 78, 367, 156}]
[{67, 234, 45, 78, 367, 23, 156}]
---------------
{234, 45, 367, 23, 89, 156}
[{234, 45, 78, 367, 89, 156}]
[{234, 45, 78, 367, 23, 89, 156}]
---------------
{67, 45, 367, 23, 89, 156}
[{67, 45, 78, 367, 89, 156}]
[{67, 45, 78, 367, 23, 89, 156}]
---------------
{234, 45, 211, 56, 89, 156}
[{234, 45, 211, 23, 56, 156} {234, 45, 78, 211, 56, 156}]
[{234, 45, 211, 23, 56, 89, 156} {234, 45, 78, 211, 56, 89, 156}]
---------------
{234, 45, 211, 23, 56, 156}
[{234, 45, 78, 211, 56, 156}]
[{234, 45, 78, 211, 23, 56, 156}]
---------------
{67, 45, 211, 56, 89, 156}
[{67, 45, 211, 23, 56, 156} {67, 45, 78, 211, 56, 156}]
[{67, 45, 211, 23, 56, 89, 156} {67, 45, 78, 211, 56, 89, 156}]
---------------
{67, 45, 211, 23, 56, 156}
[{67, 45, 78, 211, 56, 156}]
[{67, 45, 78, 21

---------------
{234, 45, 211, 23, 89, 189}
[{234, 45, 78, 211, 89, 189}]
[{234, 45, 78, 211, 23, 89, 189}]
---------------
{67, 45, 211, 56, 123, 189}
[{45, 211, 56, 89, 123, 189} {45, 211, 23, 56, 123, 189}
 {45, 78, 211, 56, 123, 189}]
[{67, 45, 211, 56, 89, 123, 189} {67, 45, 211, 23, 56, 123, 189}
 {67, 45, 78, 211, 56, 123, 189}]
---------------
{45, 211, 56, 89, 123, 189}
[{45, 211, 23, 56, 123, 189} {45, 78, 211, 56, 123, 189}]
[{45, 211, 23, 56, 89, 123, 189} {45, 78, 211, 56, 89, 123, 189}]
---------------
{45, 211, 23, 56, 123, 189}
[{45, 78, 211, 56, 123, 189}]
[{45, 78, 211, 23, 56, 123, 189}]
---------------
{67, 45, 211, 56, 89, 189}
[{67, 45, 211, 23, 56, 189} {67, 45, 78, 211, 56, 189}]
[{67, 45, 211, 23, 56, 89, 189} {67, 45, 78, 211, 56, 89, 189}]
---------------
{67, 45, 211, 23, 56, 189}
[{67, 45, 78, 211, 56, 189}]
[{67, 45, 78, 211, 23, 56, 189}]
---------------
{45, 211, 23, 56, 89, 189}
[{45, 78, 211, 56, 89, 189}]
[{45, 78, 211, 23, 56, 89, 189}]
-------------

KeyboardInterrupt: 

Results

In [ ]:
test_result